In [1]:
This tool takes a csv or an excel file and uploads it into the database as a table.
The tool is more useful where the database tool does not have a reliable 
in-house feature that can import data properly.

Step 1: GUI lets the user select the file format they want to import: csv or excel.
Step 2: User can input: 
            - login details, 
            - browse the file they want to import, 
            - specify the database and table name,
            - and hit submit.
Step 3: Tool will notify:
            - Table created successfully, if they table gets loaded successfully.
            - Pops-up an error if there is some issue with detail about the issue.
            

SyntaxError: invalid syntax (<ipython-input-1-a87a228bba51>, line 1)

In [ ]:
import sys
import pyodbc
import pandas as pd
import psycopg2
from datetime import datetime 
from pathlib import Path
import PySimpleGUI as sg
import openpyxl
import numpy as np
import os
import re

#Creaitng the connection to Greenplum server

def is_valid_path(filepath):
        if filepath and Path(filepath).exists():
            return True
        else:
            sg.popup_error("Filepath is Incorrect")
            return False
    
def display_file(filepath, filetype, sheet = None):
        if filetype == "CSV":
            df = pd.read_csv(filepath)
            pd.set_option("display.max_rows", 500)
            filename = Path(filepath).stem
            sg.popup_scrolled(df, title = filename, size= (120, 40))
        if filetype == "EXCEL":
            df = pd.read_excel(filepath, sheet_name = sheet)
            pd.set_option("display.max_rows", 500)
            filename = Path(filepath).stem
            sg.popup_scrolled(df, title = filename, size= (120, 40))

def display_table(table_name):
        df = pd.read_sql_query(sql = f'select * from {table_name} limit 10',con = conn )
            
        sg.popup_scrolled(df, title = filename, size= (120, 40))
            
            
def create_table_gp_prod(filepath, table_name, pwd, username, sheet = None , file_type = "CSV"):
        """We assign None to cur and conn because if the try clause throws an error,
        conn and cur won't be created. When finally clause runs cur and conn won't have any value. 
        So, we assign them None in the beginning"""

        conn = None
        cur = None
        connection = ""
                
        try:
            database= 'Databse Name'
            hostname= 'localhost'
            port_id =5432
            
            conn = psycopg2.connect(
                host = hostname,
                dbname = database,
                user = username,
                password = pwd,
                port = port_id)
    
            #creating a cursor
            cur = conn.cursor()
            connection = "Successful"
        except:
            sg.popup_error("Username or Password is incorrect. Try Again!")
        
        if connection == "Successful":
            table_status = ""
            
            try: 
                       
                #Importing the file and creating a dataframe called data with it
                #data = pd.read_excel(r'C:\Users\japathak\Desktop\om_policyholder.xlsx', sheet_name = 'Sheet1')
                if file_type == "CSV":
                    data = pd.read_csv(filepath)
                if file_type == "EXCEL":
                    data = pd.read_excel(filepath, sheet_name = sheet)
    
                #replacing all the ' '(single space) with Null values to drop all the row with blank values. 
                #this will remove all the rows with only blank values
                for i in data.columns:
                    data[i].replace(' ', np.nan, inplace = True)
                data.dropna(how = 'all', inplace = True)
    
                #replacing null values with ' ' to get blank values
    
                data.fillna(" ", inplace = True)
    
    
                table = f'sbx_shared.{table_name}'
    
                #Creating a dictionary to connect python datatypes and it's equivalent for postresql
                dtype = {'int64':'bigint', 'datetime64[ns]': 'date', 'float64': 'numeric',
                         'bool':'varchar(100)', 'object': 'varchar(100)','str': 'varchar(100)' }                
                    
                    
    
                #Creating a string for the create table query. It brings the column name and the it's datatype
                list = []
                for col in data.columns:
                    #list.append(col.replace(" ", "_") +" "+ dtype[str(data[col].dtypes)])
                    if data[col].dtype == 'object':
                        try:
                            data[col] = pd.to_datetime(data[col])
                            list.append(re.sub(r'[^a-zA-Z0-9\s]', '', col).strip().replace(" ", "_") +" "+ dtype[str(data[col].dtypes)])
    
                        except Exception:
    
                            list.append(re.sub(r'[^a-zA-Z0-9\s]', '',col).strip().replace(" ", "_") +" "+ dtype[str(data[col].dtypes)]) 
                    else:
                        try:
                            list.append(re.sub(r'[^a-zA-Z0-9\s]', '',col).replace(" ", "_") +" "+ dtype[str(data[col].dtypes)]) 
                        except Exception:
                            list.append(re.sub(r'[^a-zA-Z0-9\s]', '',col).replace(" ", "_") +" "+ "Varchar(50)")
    
                col_string = str(list).replace("'","").replace("[", "(").replace("]", ")").replace("?COLUMN?", "COLUMN_NAME")
    
                #This part connect the drop and replace string with the columns string
            
                create_string = f'CREATE TABLE {table} '
                create_query = create_string + col_string
    
                #Executing the drop and create table query
                cur.execute(create_query)
    
                conn.commit()
                table_status = "New"
            except Exception as err:
                print(err)
                sg.popup_error("Table already exists. \nPlease change the name and try again.")
                sg.popup_error(err)
            
            if table_status == "New":
                                
                try:
                    if file_type == "CSV":
    
                        output = filepath+'_1'
                        try: 
                            os.remove(output)
                        except:
                            pass
                        data.to_csv(output, index = False)
    
                        f = open(output, 'r', encoding='utf-8' )
                        sql = f"COPY {table} FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                        cur.copy_expert(sql = sql, file = f)
                        conn.commit()
    
                        f.close()
                        try: 
                            os.remove(output)
                        except:
                            pass
    
                    if file_type == "EXCEL":
    
                    #following code creates insert statement for each row in the table. Some cell values might have ' (Apostrophe) in it
                    #like name O'brien. That could throw an error. In that case we are replacing ' with ".
                        list1 = []
                        for row in range(0, data.shape[0]):
                            list2 = []
                            for cell in data.values[row]:
    
                                if type(cell) is int or type(cell) is float:
                                    list2.append(cell)
    
                                else:
                                    list2.append(cell.replace("'", '"'))
    
                            list1.append(list2) 
                        for row in range(0, len(list1)):
                            cur.execute(f"insert into {table} VALUES {tuple(list1[row])}")
                            conn.commit()
    
    
                    sg.popup_no_titlebar("Table is Created in Greenplum!")
    
    
    
                
                                            
                #printing the error if there was an exception   
                except Exception as err:
                    print(err )
                    print("Something went wrong while creating the table")
                    sg.popup_error("There was an error. Please try again or reach out to Jay Pathak.")
        
                #Closing cur and conn if they were open    
                finally: 
                    if cur is not None:
                        cur.close()
                    if conn is not None:
                        conn.close()
                        
def create_table_gp_edw_uat(filepath, table_name, pwd, username, sheet = None , file_type = "CSV"):
        
        """We assign None to cur and conn because if the try clause throws an error,
        conn and cur won't be created. When finally clause runs cur and conn won't have any value. 
        So, we assign them None in the beginning"""

        conn = None
        cur = None
        connection = ""
        
        try:
            database= 'EDW_UAT'
            hostname= 'gp-test'
            port_id =5432
            
            conn = psycopg2.connect(
                host = hostname,
                dbname = database,
                user = username,
                password = pwd,
                port = port_id)

            #creating a cursor
            cur = conn.cursor()
            connection = "Successful"
        except:
            sg.popup_error("Username or Password is incorrect. Try Again!")
        
        if connection == "Successful":
            table_status = ""
            try: 
               
                #Importing the file and creating a dataframe called data with it
                #data = pd.read_excel(r'C:\Users\japathak\Desktop\om_policyholder.xlsx', sheet_name = 'Sheet1')
                if file_type == "CSV":
                    data = pd.read_csv(filepath)
                if file_type == "EXCEL":
                    data = pd.read_excel(filepath, sheet_name = sheet)

                #replacing all the ' '(single space) with Null values to drop all the row with blank values. 
                #this will remove all the rows with only blank values
                for i in data.columns:
                    i = i.strip()
                    data[i].replace(' ', np.nan, inplace = True)
                data.dropna(how = 'all', inplace = True)

                #replacing null values with ' ' to get blank values

                data.fillna(" ", inplace = True)


                table = f'sbx_shared.{table_name}'

                #Creating a dictionary to connect python datatypes and it's equivalent for postresql
                dtype = {'int64':'bigint', 'datetime64[ns]': 'date', 'float64': 'numeric',
                         'bool':'varchar(100)', 'object': 'varchar(100)','str': 'varchar(100)' }

                #Creating a string for the create table query. It brings the column name and the it's datatype
                list = []
                for col in data.columns:
                    #list.append(col.replace(" ", "_") +" "+ dtype[str(data[col].dtypes)])
                    if data[col].dtype == 'object':
                        try:
                            data[col] = pd.to_datetime(data[col])
                            list.append(re.sub(r'[^a-zA-Z0-9\s]', '', col).strip().replace(" ", "_") +" "+ dtype[str(data[col].dtypes)])
    
                        except Exception:
    
                            list.append(re.sub(r'[^a-zA-Z0-9\s]', '',col).strip().replace(" ", "_") +" "+ dtype[str(data[col].dtypes)]) 
                    else:
                        try:
                            list.append(re.sub(r'[^a-zA-Z0-9\s]', '',col).replace(" ", "_") +" "+ dtype[str(data[col].dtypes)]) 
                        except Exception:
                            list.append(re.sub(r'[^a-zA-Z0-9\s]', '',col).replace(" ", "_") +" "+ "Varchar(50)")

                col_string = str(list).replace("'","").replace("[", "(").replace("]", ")").replace("?COLUMN?", "COLUMN_NAME")

                #This part connect the drop and replace string with the columns string
            
                
                create_string = f'CREATE TABLE {table} '
                create_query = create_string + col_string

                #Executing the drop and create table query
                cur.execute(create_query)

                conn.commit()
                table_status = "New"
            except:
                sg.popup_error("Table already exists. \nPlease change the name and try againnn.")
            
            if table_status == "New":
                try:
                    if file_type == "CSV":
                        
                        output = filepath+'_1'
                        data.to_csv(output, index = False)

                        f = open(output, 'r', encoding='utf-8')
                        sql = f"COPY {table} FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                        cur.copy_expert(sql = sql, file = f)
                        conn.commit()

                        f.close()
                        try: 
                            os.remove(output)
                        except:
                            pass

                    if file_type == "EXCEL":

                    #following code creates insert statement for each row in the table. Some cell values might have ' (Apostrophe) in it
                    #like name O'brien. That could throw an error. In that case we are replacing ' with ".
                        list1 = []
                        for row in range(0, data.shape[0]):
                            list2 = []
                            for cell in data.values[row]:

                                if type(cell) is int or type(cell) is float:
                                    list2.append(cell)

                                else:
                                    list2.append(cell.replace("'", '"'))

                            list1.append(list2) 
                        for row in range(0, len(list1)):
                            cur.execute(f"insert into {table} VALUES {tuple(list1[row])}")
                            conn.commit()


                    sg.popup_no_titlebar("Table is Created in Greenplum!")

    
    
                        

                #printing the error if there was an exception   
                except Exception as err:
                    print(err )
                    print("Something went wrong while creating the table")
                    sg.popup_error("There was an error. Please try again or reach out to Jay Pathak.")

                #Closing cur and conn if they were open    
                finally: 
                    if cur is not None:
                        cur.close()
                    if conn is not None:
                        conn.close()
                
def insert_into_gp_prod(filepath, table_name, username, pwd):
        """We assign None to cur and conn because if the try clause throws an error,
        conn and cur won't be created. When finally clause runs cur and conn won't have any value. 
        So, we assign them None in the beginning"""

        conn = None
        cur = None
        connection = ""
        
        try:
            database= 'EDW_PROD'
            hostname= 'gp-prod.safetyinsurancenet.com'
            port_id =5432
            
            conn = psycopg2.connect(
                host = hostname,
                dbname = database,
                user = username,
                password = pwd,
                port = port_id)

            #creating a cursor
            cur = conn.cursor()
            connection = "Successful"
        except:
            sg.popup_error("Username or Password is incorrect. Try Again!")
        
        if connection == "Successful":
            try:
    
                #Importing the file and creating a dataframe called data with it
                
                data = pd.read_csv(filepath)
    
    
                #replacing all the ' '(single space) with Null values to drop all the row with blank values. 
                #this will remove all the rows with only blank values
                for i in data.columns:
                    data[i].replace(' ', np.nan, inplace = True)
                
                data.dropna(how = 'all', inplace = True)
    
                #replacing null values with ' ' to get blank values
    
                data.fillna(" ", inplace = True)
    
    
                table = f'sbx_shared.{table_name}'
    
                output = filepath+'_1'
                try: 
                    os.remove(output)
                except:
                    pass
                data.to_csv(output, index = False)
    
                f = open(output, 'r', encoding='utf-8')
                sql = f"COPY {table} FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                cur.copy_expert(sql = sql, file = f)
                conn.commit()
    
                f.close()
                try: 
                    os.remove(output)
                except:
                    pass
                sg.popup_no_titlebar("Rows inserted to the table successfully!")
                
            #printing the error if there was an exception   
            except Exception as err:
                print(err )
                print("Something went wrong while creating the table")
                sg.popup_error("There was an error. Please try again or reach out to Jay Pathak.")

            #Closing cur and conn if they were open    
            finally: 
                if cur is not None:
                    cur.close()
                if conn is not None:
                    conn.close()   
    
    
    
    
#This function opens the main window if user selects CSV
def main_window_csv():
    #Creating the layout for main window
    layout = [
        [sg.Text("Username: "), sg.Input(key = "username")],
        [sg.Text("Password: "), sg.InputText('',key = "pwd", password_char = '*')],
        [sg.Text("Input File: "), sg.Input(key = "-IN-"), sg.FileBrowse(file_types = (("CSV FILES", "*.csv"),), button_text = "Search File")],
        [sg.Text("Note: Table will be created in SBX_SHARED for DB you select.")],
        [sg.Text("Select Database: ")],
        [sg.Listbox(values = ["GP-Prod", "GP-UAT"], key = "database", size = (20,3))],
        [sg.Text("Table name: "), sg.Input(key = "tablename")],
        [sg.Exit(), sg.Button("Back"), sg.Button("Display File"), sg.Button("Create Table"),
        sg.T("Created by: Jay Pathak", justification = 'r', s = 60, font = ('Arial', 8))]
        
    ]
    
    window_title = "Create Table: Select Your CSV File"
    window = sg.Window(window_title, layout)
    
    #Creating the scenarios and there consecuent actions
    while True:
        event, values = window.read()
        #print(event, values)
        if event in (sg.WINDOW_CLOSED, "Exit"):
            break
        
        if event == "Back":
            break
        
        if event == "Display File":
            try:
                if is_valid_path(values["-IN-"]):
                    display_file(filepath = values["-IN-"])
            except Exception as err:
                print('display_file_error: ', err)
                break
        if event == "Create Table":
            try:
                if is_valid_path(values["-IN-"]):
                    if values["database"] == []:
                        sg.popup_error("You must select a database")
                    if values["tablename"] == "":
                        sg.popup_error("You must enter a table name")
                    
                    else:
                        if values["database"] == ["GP-Prod"]:
                            create_table_gp_prod(
                            filepath = values["-IN-"],
                            table_name = values["tablename"],
                            username = values["username"],
                            pwd = values["pwd"])
                        elif values["database"] == ["GP-UAT"]:
                            create_table_gp_edw_uat(
                            filepath = values["-IN-"],
                            table_name = values["tablename"],
                            username = values["username"],
                            pwd = values["pwd"])
                        else:
                            print("Error in databases while creating table in csv")
                        
            except Exception as err:
                print('create_table_error: ', err)
                sg.popup_error("There was an error. Please reach out to Jay Pathak.")
                
        
    window.close()


    
    
#This function displays the main window if user selects Excel format
def main_window_excel():
    layout = [
        [sg.Text("Username: "), sg.Input(key = "username")],
        [sg.Text("Password: "), sg.InputText('', key = "pwd", password_char = '*')],
        [sg.Text("Input File: "), sg.Input(key = "-IN-"), sg.FileBrowse(file_types = (("EXCEL FILES", "*.xlsx"),), button_text = "Search File")],
        [sg.Text("Enter Sheet Name: "), sg.Input(key = "sheet")],
        [sg.Text("Note: Table will be created in SBX_SHARED for DB you select.")],
        [sg.Text("Select Database: ")],
        [sg.Listbox(values = ["GP-Prod", "GP-UAT"], key = "database", size = (20,3))],
        [sg.Text("Table name: "), sg.Input(key = "tablename")],
        [sg.Exit(), sg.Button("Back"), sg.Button("Display File"), sg.Button("Create Table"), 
        sg.T("Created by: Jay Pathak", justification = 'r', s = 60, font = ('Arial', 8))]
        
    ]
    
    window_title = "Create Table: Select Your Excel File"
    window = sg.Window(window_title, layout)
    
    while True:
        event, values = window.read()
        #print(event, values)
        if event in (sg.WINDOW_CLOSED, "Exit"):
            break
        
        if event == "Back":
            break
        
        if event == "Display File":
            try:
                if is_valid_path(values["-IN-"]):
                    if values["sheet"] not in pd.ExcelFile(values["-IN-"]).sheet_names:
                        sg.popup_error("Sheet Name is Incorrect")
                    else:
                        display_file(filepath = values["-IN-"], filetype = "EXCEL", sheet = values["sheet"])
            except Exception as err:
                print('display_file_error: ', err)
                break 
        if event == "Create Table":
                  
            try:
                if is_valid_path(values["-IN-"]):
                    if values["sheet"] not in pd.ExcelFile(values["-IN-"]).sheet_names:
                        sg.popup_error("Sheet Name is Incorrect")
                    if values["database"] == []:
                        sg.popup_error("You must select a database")
                    if values["tablename"] == "":
                        sg.popup_error("You must enter a table name")
                    
                    else:
                        sg.popup_no_wait("Your request is being processed...\nCan take a few minutes (depends on file size).\nYou will be notified once done!"
                                         , title = "Notification")
                        if values["database"] == ["GP-Prod"]:
                            create_table_gp_prod(
                            filepath = values["-IN-"],
                            table_name = values["tablename"],
                            username = values["username"],
                            pwd = values["pwd"], 
                            sheet = values["sheet"],
                            file_type = "EXCEL")
                        elif values["database"] == ["GP-UAT"]:
                            create_table_gp_edw_uat(
                            filepath = values["-IN-"],
                            table_name = values["tablename"],
                            username = values["username"],
                            pwd = values["pwd"])
                        else:
                            print(values["database"])
                                    
            except Exception as err:
                print('create_table_error: ', err)
                sg.popup_no_wait("There was an error. Please try again or reach out to Jay Pathak.")
                break
        
    window.close()


#This function opens the main window if user selects Insert from CSV file
def main_window_insert():
    #Creating the layout for main window
    layout = [
        [sg.Text("Username: "), sg.Input(key = "username")],
        [sg.Text("Password: "), sg.InputText('',key = "pwd", password_char = '*')],
        [sg.Text("Input File: "), sg.Input(key = "-IN-"), sg.FileBrowse(file_types = (("CSV FILES", "*.csv"),), button_text = "Search File")],
        [sg.Text("Note: Rows will be inserted in SBX_SHARED for DB and table name you select.")],
        [sg.Text("Select Database: ")],
        [sg.Listbox(values = ["GP-Prod", "GP-UAT"], key = "database", size = (20,3))],
        [sg.Text("Table name: "), sg.Input(key = "tablename")],
        [sg.Exit(), sg.Button("Back"), sg.Button("Insert Rows"),
        sg.T("Created by: Jay Pathak", justification = 'r', s = 60, font = ('Arial', 8))]
        
    ]
    
    window_title = "Insert Rows Into Table: Select Your CSV File"
    window = sg.Window(window_title, layout)
    
    #Creating the scenarios and there consecuent actions
    while True:
        event, values = window.read()
        #print(event, values)
        if event in (sg.WINDOW_CLOSED, "Exit"):
            break
        
        if event == "Back":
            break
        
        if event == "Insert Rows":
            try:
                if is_valid_path(values["-IN-"]):
                    if values["database"] == []:
                        sg.popup_error("You must select a database")
                    if values["tablename"] == "":
                        sg.popup_error("You must enter a table name")
                    
                    else:
                        if values["database"] == ["GP-Prod"]:
                            insert_into_gp_prod(
                            filepath = values["-IN-"],
                            table_name = values["tablename"],
                            username = values["username"],
                            pwd = values["pwd"])
                        elif values["database"] == ["GP-UAT"]:
                            insert_into_gp_edw_uat(
                            filepath = values["-IN-"],
                            table_name = values["tablename"],
                            username = values["username"],
                            pwd = values["pwd"])
                        else:
                            print("Error in databases while creating table in csv")
                        
            except Exception as err:
                print('create_table_error: ', err)
                sg.popup_error("There was an error. Please reach out to Jay Pathak.")
                
        
    window.close()
    

theme = "DarkTeal10"
font_family = "Arial"
font_size = 14
sg.theme(theme)
sg.set_options(font = (font_family, font_size))


#This functions creates the very first window that lets user choose either csv or excel
def main_window():
    layout =[
        [sg.Text("Select the file type you want to create the table from: ")],
        [sg.Button("CSV")],
        [sg.Button("EXCEL")],
        [sg.Button("Insert from CSV file")],
        [sg.Exit(), sg.T("Created by: Jay Pathak", justification = 'r', s = 60, font = ('Arial', 8), text_color = 'white')]
               
    ]
    
    title = "Create table: Select your file type"
    window = sg.Window(title, layout)
    for i in range(1):
        if is_valid_path(r'S:\Underwriting\Actuarial\Yellow\Data Governance\Business Intelligence\Jay\File Importer\Jay doc.docx'):
            pass
        else:
            window.close()
    
    while True:
        event, values = window.read()
        
        if event in ("Exit", sg.WINDOW_CLOSED):
            break
        if event == "CSV":
            main_window_csv()
        if event == "EXCEL":
            main_window_excel()
        if event == "Insert from CSV file":
            main_window_insert()
    window.close()
main_window()